In [1]:
# Initialize Otter
import otter
grader = otter.Notebook("covid_part1.ipynb")

# Final Project: COVID-19 Dataset
## Exploring COVID-19 Data through Modeling
## Due Date: Wednesday, November 17th, 11:59 PM
## Collaboration Policy

Data science is a collaborative activity. While you may talk with other groups about
the project, we ask that you **write your solutions within your own group**. If you do
discuss the assignments with others outside of your group please **include their names** at the top
of your notebook.


## This Assignment

In this final project, we will investigate COVID-19 data over the past year. This data contains information about COVID-19 case counts, mortalities, vaccination rates, and various other metadata that can assist in modeling various aspects of COVID-19.

Through this final project, you will demonstrate your experience with:
* Data cleaning and EDA using Pandas
* Unsupervised and supervised learning techniques
* Visualization


## Goal

Model and analyze the temporal evolution of COVID-19 mortalities or cases using one unsupervised and one supervised technique of your choice. Interpret your models' results through visualizations, and draw insightful conclusions about the modeling of COVID-19 data.

Recall that we studied linear and logistic regression, decision trees, random forests as part of supervised learning (with labels) and clustering, PCA as part of unsupervised learning (without labels). You are free to use any methods that you find suitable to answer the problem at hand.

In [ ]:
# Run this cell to set up your notebook
import numpy as np
from geopy import *
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.cluster import *
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from scipy.stats import pearsonr
import re

cases = pd.read_csv('data/time_series_covid19_confirmed_US.csv') # https://github.com/CSSEGISandData/COVID-19/blob/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv
vaccinations = pd.read_csv('data/people_vaccinated_us_timeline.csv') # https://raw.githubusercontent.com/govex/COVID-19/master/data_tables/vaccine_data/us_data/time_series/people_vaccinated_us_timeline.csv
counties = pd.read_csv('data/co-est2020.csv', encoding='latin-1') # https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/counties/totals/co-est2020.csv
mask_use = pd.read_csv('data/mask-use-by-county.csv') # https://github.com/nytimes/covid-19-data/blob/master/mask-use/mask-use-by-county.csv

<br/><br/><br/>

---

<br/><br/><br/>

## Question 0: Basic Time Series Modeling

We will introduce a few basic modeling techniques using temporally correlated data that you are free to adapt or improve throughout your modeling process if you wish. Answer these questions in their respective cells below.

### Question 0a
Suppose $\tilde{x} = [1, x_{t}]$ represents the input to a machine learning function $f_\theta: \mathbb{R}^{2} \rightarrow \mathbb{R}$, and we wish to predict $x_{t + 1}$ such that $f_\theta(\tilde{x}) \approx x_{t+1}$. You may assume that $t = 0$ corresponds to the start of time. 

<!-- BEGIN QUESTION -->

i) Is this an example of supervised or unsupervised learning?
<!--
BEGIN QUESTION
name: q0ai
points: 2
manual: True
-->

This would be an example of supervised learning since we are aware of what we want to predict

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

ii) Write the function $x_{t+1}$ explicitly in terms of the model parameters $\theta = [\theta_0, \theta_1]$ and $x_t$ assuming $f$ represents a linear model.
<!--
BEGIN QUESTION
name: q0aii
points: 2
manual: True
-->

$x_{t+1}$ ≈ $𝑓_{𝜃}(𝑥̃)$ and $𝑓_{𝜃}(𝑥̃)$ is the linear model output for 𝑥̃. 

From the linear modeling lecture, $𝑓_{𝜃}(𝑥̃)$ = $𝜃^{T}$𝑥̃ , so we can conclude that: $𝑓_{𝜃}(𝑥̃)$ ≈ $x_{t+1}$ = $𝜃_{0}$ + $𝜃_{1}x_{t}$

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

iii) Suppose we obtain an optimal $\hat{\theta} = [\hat{\theta}_0, \hat{\theta}_1]$. Derive $x_{t+1}$ in terms of $x_0$, $\hat{\theta}_0$ and $\hat{\theta}_1$.

<!--
BEGIN QUESTION
name: q0aiii
points: 4
manual: True
-->

From the expression $x_{t+1}$ = $𝜃_{0}$ + $𝜃_{1}x_{t}$ ,   we can find, by using  $\hat{𝜃}_{0}$ and $\hat{𝜃}_{1}$ , the expression of $x_{t+1}$. 

By calculating $x_{1}$, $x_{2}$, and so on, and finding the pattern, one finds that:


$$x_{t+1} = \sum_{i=0}^t \hat{𝜃}_{1}^{i} \hat{𝜃}_{0} + \hat{𝜃}_{1}^{t+1} x_{0}$$

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

iv) Suppose we add $k - 1$ more features such that our feature vector $\tilde{x}$ contains $k$ timesteps of past and current data. Describe how we could select an appropriate $k$.

<!--
BEGIN QUESTION
name: q0aiv
points: 2
manual: True
-->

In this case, k is a hyperparameter. It’s usually up to us what k value we decide to choose but the best k value can be determined through cross-validation. With cross-validation, the k value that leads to the best model performance can be found without overfitting or underfitting the training data.

<!-- END QUESTION -->



<br/><br/><br/>

---

<br/><br/><br/>

## Question 1: Data Cleaning

### Question 1a

Investigate the number of missing or null values in `cases` and `vaccinations`. Which one column contains the *most* null values from both of these tables?

<!--
BEGIN QUESTION
name: q1a
points: 2
-->

In [ ]:
cases.head()

In [ ]:
v = vaccinations.isna().sum().sort_values(ascending=False).head(1)
c = cases.isna().sum().sort_values(ascending=False).head(1)
most_null_value_col = "People_Partially_Vaccinated"
most_null_value_col

In [ ]:
grader.check("q1a")

### Question 1b

Impute the null values in *all* the datasets with zero values or empty strings where appropriate.

<!--
BEGIN QUESTION
name: q1b
points: 2
-->

In [ ]:
vaccinations = vaccinations.fillna(0)
cases = cases.fillna(0)

In [ ]:
grader.check("q1b")

### Question 1c

Add a column to `cases` named `median_increase` that describes the median daily increase in the number of cases over the time period January 23, 2020 to September 12, 2021. In other words, calculate the increase in cases day-to-day from January 22nd to 23rd, 23rd to 24th, 24th to 25th, and so on; then, find the median of the number of increase in cases over that time period for all counties.

*Hint*: Try not to use a `for` loop.

<!--
BEGIN QUESTION
name: q1c
points: 2
-->

In [ ]:
cases_in_time = cases.loc[:, '1/23/20' : '9/12/21']
diff = cases_in_time.diff(axis = 1).fillna(0)
median_diff = np.median(diff, axis = 1)
cases['median_increase'] = median_diff

In [ ]:
grader.check("q1c")

### Question 1d

Generate a valid FIPS code for the `counties` table.

*Hint*: Refer to [this](https://transition.fcc.gov/oet/info/maps/census/fips/fips.txt) guide on FIPS codes.

<!--
BEGIN QUESTION
name: q1d
points: 2
-->

In [ ]:
state_code = counties['STATE']
sc_str = state_code.map('{:02}'.format).astype(str)
county_code = counties['COUNTY']
cc_str = county_code.map('{:03}'.format).astype(str)
FIPS = sc_str + cc_str
counties['FIPS'] = FIPS


In [ ]:
grader.check("q1d")

### Question 1e

Merge the `counties`, `cases`, and `mask_use` tables on an appropriate primary key to generate county-wise data.

<!--
BEGIN QUESTION
name: q1e
points: 2
-->

In [ ]:
counties['PK'] = counties['FIPS'].astype(int)
cases['PK'] = cases['FIPS'].astype(int)

county_data = pd.merge(pd.merge(counties, cases, on = 'PK', how = 'inner'), 
                       mask_use, left_on='PK', right_on='COUNTYFP', how = 'inner')

county_data = county_data.drop(columns = ['PK', 'FIPS_y'])
county_data = county_data.rename({'FIPS_x': 'FIPS'}, axis = 'columns')
cases = cases.drop(columns = 'PK')
counties = counties.drop(columns= 'PK')
                                          

In [ ]:
grader.check("q1e")

<!-- BEGIN QUESTION -->

### Question 1f

Generate the population by state using `counties` using the population estimate in 2020 (i.e. `POPESTIMATE2020`). Remark on any inconsistencies and propose a solution. 
<!--
BEGIN QUESTION
name: q1f
points: 2
manual: True
-->

In [ ]:
counties[counties['STNAME'] == counties['CTYNAME']][['STNAME', 'CTYNAME', 'POPESTIMATE2020']]

If we look closely at the counties table, we can observe that for each state, the rows contain the total population of that state. If both the 'STNAME' and 'CTYNAME' columns have the names of the states, then the 'POPESTIMATE2020' column will have the total population for the state in 2020. This means that if we were to apply functions like groupby to the table, then the final results will be incorrect. Results will likely be 2x the actual population and won't be able to generate accurate population counts. Also, there are 52 rows, despite there being 50 states, because District of Columbia has two rows dedicated to it. 

Possible solutions I would recommend include: dropping the two rows for District of Columbia and dropping 'CTYNAME' column. 

<!-- END QUESTION -->



<br/><br/><br/>

---

<br/><br/><br/>

## Question 2: Guided EDA

<!-- BEGIN QUESTION -->

### Question 2a

Create a visualization that shows the median increase in number of cases for the 50 counties with the greatest median increase over January 23, 2020 to September 12, 2021. Make sure to include the name of state in which each county is located since county names are not necessarily unique.
<!--
BEGIN QUESTION
name: q2a
points: 3
manual: True
-->

In [ ]:
p2a = cases[['Combined_Key', 'median_increase']].sort_values(by='median_increase', ascending=False).head(50)
plt.figure(figsize = (10, 10))
plt.barh(y='Combined_Key', width='median_increase', data = p2a)
plt.xlabel('Median Increase')
plt.ylabel('Combined_Key')
plt.title('Top 50 Counties in the US by Median Increase in Covid 19')


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2b

Create a visualization that shows the overall county-wise distribution of the number of COVID-19 cases per capita across the United States as a function of time. For all questions that reference population, use the population estimates from 2020.
<!--
BEGIN QUESTION
name: q2b
points: 4
manual: True
-->

In [ ]:
counties.head()

In [ ]:
#Taking a time frame of 9 months from 1st January 2021 to 12th September 2021 
#We make cases of covid19 per capita
daily_cases = county_data.loc[:,'1/1/21':'9/12/21']
pop_per_county = county_data['POPESTIMATE2020']
per_capita = daily_cases.div(pop_per_county, axis=0)

#boxplot
ax = sns.boxplot(data=per_capita, showfliers=False)
plt.title('Overall County-Wise Distribution of COVID-19 Cases per Capita Over Time')
plt.ylabel('COVID-19 Cases per Capita')
plt.xlabel('Date');
plt.rcParams['figure.figsize'] = (20, 12)
plt.rcParams['font.size'] = 18
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
                        

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2c

Create a visualization that shows states' increases in the number of COVID-19 cases per capita from September 3rd to September 12th, sorted from least to greatest increases. 

<!--
BEGIN QUESTION
name: q2c
points: 3
manual: True
-->

In [ ]:
p2c = cases.groupby('Province_State').sum()
statePopEst = counties.groupby('STNAME').sum()[['POPESTIMATE2020']]
p2c = p2c.merge(statePopEst, left_index =True, right_on = 'STNAME')
p2c = p2c.loc[:, '9/3/21': '9/12/21'].div(p2c['POPESTIMATE2020'], axis = 0)
p2c = (p2c['9/12/21'] - p2c['9/3/21']).sort_values(ascending=False)
plt.figure(figsize=(10,10))
plt.xlabel('Increase in number of cases per capita from 9/3/21 - 9/12/21')
plt.ylabel('State')
plt.title('States ordered by the increase in number of cases from 9/3/21 - 9/12/21')
p2c.plot.barh()



<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2d

Create a plot using a technique of your choice that visualizes the relationship between:

a) the number of COVID-19 cases as a function of time for the Mobile, Tarrant, Cook, and San Fransisco counties located in Alabama, Texas, Illinois, and California.

b) the frequency of never, rarely or sometimes wearing a mask in the respective counties

*Hint*: You may find a description of the data [here](https://github.com/CSSEGISandData/COVID-19/blob/846fa9458cc2a8904d2492d14d3e6b4f399ec027/csse_covid_19_data/csse_covid_19_time_series/README.md).

<!--
BEGIN QUESTION
name: q2d
points: 3
manual: True
-->

In [ ]:
p2d = cases.where(cases['Admin2'].isin(['Mobile', 'Tarrant', 'Cook', 'San Francisco'])).dropna()
p2da = p2d.where(p2d['Province_State'].isin(['Alabama', 'Texas', 'Illinois', 'California'])).dropna().set_index('Admin2')
p2da = p2da.loc[:, '1/22/20':'9/12/21'].T
p2da.index = pd.to_datetime(p2da.index)
p2db = p2d.merge(mask_use, left_on='FIPS', right_on='COUNTYFP')
p2db['N/R/S'] = p2db['NEVER'] + p2db['RARELY'] + p2db['SOMETIMES'] 

figure, axes = plt.subplots(2, 1)
figure.tight_layout()
p2da.plot(figsize=(7, 10), ax=axes[0])
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Number of cases')
axes[0].set_title('Number of COVID 19 Cases over time')

axes[1] = plt.barh(y='Admin2', width='N/R/S', data=p2db)
axes[1] = plt.title('Frequency of Never/Rarely/Sometimes Wearing a Mask for 4 Counties')

axes[1] = plt.xlabel('Frequency of Never/Rarely/ Sometimes Wearing a Mask')
axes[1] = plt.ylabel('County')


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2e

Visualize and describe the spatial relationship between the location and the following quantities over the continguous United States:

a) the number of most recent COVID-19 cases per capita on September 12, 2021

b) the frequency of never, rarely or sometimes wearing a mask

*Hint*: Use `plotly` to generate a heatmap on a geographical plot of the United States!

<!--
BEGIN QUESTION
name: q2e
points: 5
manual: True
-->

In [ ]:
popEst = counties[['POPESTIMATE2020', 'FIPS']]
popEst['FIPS'] = popEst['FIPS'].astype(int)

In [ ]:
from urllib.request import urlopen
import json
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    countiesjson = json.load(response)

p2e = cases.merge(popEst, on='FIPS')
p2e = p2e.merge(mask_use, left_on = 'FIPS', right_on='COUNTYFP')
p2e.loc[:, '1/22/20':'9/12/21'] = p2e.loc[:, '1/22/20':'9/12/21'].div(p2e['POPESTIMATE2020'], axis=0)
p2e['N/R/S'] = p2e['NEVER'] + p2e['RARELY'] + p2e['SOMETIMES']
p2e['FIPS'] = p2e['FIPS'].astype(int).astype(str).str.zfill(5)
p2e = p2e[~p2e['Province_State'].isin(['Alaska', 'Hawaii'])]

fig = make_subplots(rows=1, cols=2, specs=[[{"type": "choropleth"},{"type": "choropleth"}],])
fig.add_trace(go.Choropleth(geojson=countiesjson, colorscale="Viridis", locations=p2e['FIPS'], z=p2e['N/R/S'], colorbar_x=0.45,
                           name='Frequency of Never/Rarely/Sometimes wearing a Mask'), row=1, col=1)
fig.add_trace(go.Choropleth(geojson=countiesjson, colorscale="Viridis", locations=p2e['FIPS'], z=p2e['9/12/21']-p2e['9/11/21'], zmin=(p2e['9/12/21']-p2e['9/11/21']).min(),
                           zmax=(p2e['9/12/21']-p2e['9/11/21']).max()-.001, name='Cases on 9/12/21 per capita'), row=1, col=2)
fig.update_geos(fitbounds="locations", visible=False)
fig.update_layout(title='Frequency of Never/Rarely/Sometimes wearing a Mask vs Number of COVID cases per capita on 9/12/21')
fig.show()

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 2f

Describe and interpret the relationships in the plots above. What conclusions can be drawn about the COVID-19 dataset from these plots?

<!--
BEGIN QUESTION
name: q2f
points: 3
manual: True
-->

Looking at the two heatmaps above, there appears to be a correlation between amount of covid cases per capita and frequency of mask usage. From the plots, we can infer that lower mask usage is related to higher covid case rates. However, the plot we generated in 2d appears to suggest something different. Despite Mobile's lower mask usage , their number of covid cases was low, similar to San Francisco's covid rate - despite San Francisco's higher mask usage. We can also observe that the growth rate of covid cases in counties displayed a normal distribution with a slight skew right. 

<!-- END QUESTION -->



<br/><br/><br/>

---

<br/><br/><br/>

## Question 3: Guided Unsupervised Exploration


<!-- BEGIN QUESTION -->

### Question 3a

Using SVD, decompose the standardized matrix $X$ that contains the standardized current proportion of fully vaccinated and partially vaccinated, cases per capita, and mask usage for every state as of the most recent day in the data.

*Hint*: The matrix $X$ should have rows that describe location.

<!--
BEGIN QUESTION
name: q3a
points: 3
manual: True
-->

In [ ]:
# Get the latest possible date with vaccination info and cases info
vaccinations_latest = vaccinations[vaccinations['Date'] == '2021-09-12']

# Merge county_data and vaccination data
wanted_cols = ['STNAME', 'POPESTIMATE2020', 'People_Fully_Vaccinated', 'People_Partially_Vaccinated', '9/12/21',
               'NEVER' , 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS', 'Long__y', 'Lat_y']
merged = pd.merge(county_data, vaccinations_latest, how='inner', left_on="STNAME", right_on="Province_State")[wanted_cols]

# Calculate the absolute number of mask usage rather than proportion
for col_name in ('NEVER' , 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS'):
    merged["{}_ABS".format(col_name)] = merged[col_name] * merged["POPESTIMATE2020"]

# Group by state. Everything should be added except for vaccination info and long/lat since that info is by state
by_state = merged.groupby("STNAME").agg({"POPESTIMATE2020": "sum", "People_Fully_Vaccinated": "first", "People_Partially_Vaccinated": "first",
                              "9/12/21": "sum", "NEVER_ABS": "sum", "RARELY_ABS": "sum", "SOMETIMES_ABS": "sum",
                              "FREQUENTLY_ABS": "sum", "ALWAYS_ABS": "sum", "Long__y": "first", "Lat_y": "first"})

# Now that everything's summed, we can calculate mask usage proportions again
for col_name in ('NEVER' , 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS'):
    col_name_abs = col_name + '_ABS'
    by_state[col_name] = by_state[col_name_abs] / by_state['POPESTIMATE2020']

# Calculate the cases per capita
by_state['cases_per_capita'] = by_state['9/12/21'] / by_state['POPESTIMATE2020']

# Calculate proportion of vaccinations
by_state['fully_vaccinated_proportion'] = by_state['People_Fully_Vaccinated'] / by_state['POPESTIMATE2020']
by_state['partially_vaccinated_proportion'] = by_state['People_Partially_Vaccinated'] / by_state['POPESTIMATE2020']

# Select only the columns we want
standardized_matrix = by_state[['fully_vaccinated_proportion', 'partially_vaccinated_proportion', 'cases_per_capita', 'NEVER', 'RARELY', 'SOMETIMES', 'FREQUENTLY', 'ALWAYS']]



In [ ]:
X = standardized_matrix
X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X = (X - X_mean) / X_std
u, s, vt = np.linalg.svd(X, full_matrices = False)


<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 3b

Visualize the top 2 principal components and label the data points with the corresponding location. Color each data points based on the number of cases per capita in the location to which it corresponds.

<!--
BEGIN QUESTION
name: q3b
points: 3
manual: True
-->

In [ ]:
X_pcs = pd.DataFrame(np.matmul(u, np.diag(s)))
X_pcs = X_pcs.rename(columns = {0: 'pc1', 1: 'pc2'})

#Add back columns for visualization
X_pcs['cases_per_capita'] = list(standardized_matrix['cases_per_capita'])
X_pcs['state'] = list(by_state.index)

#Plot
sns.set(rc = {'figure.figsize': (15, 10)})

ax = sns.scatterplot(data = X_pcs, x = 'pc1', y = 'pc2', palette = 'RdBu_r', hue = 'cases_per_capita', s = 50)
for line in range(0, X_pcs.shape[0]): 
    ax.text(X_pcs.pc1[line]+0.1, X_pcs.pc2[line],
    X_pcs.state[line], horizontalalignment='left', size='medium', color='black')
plt.legend(bbox_to_anchor=(1.21, 1), title='Cases Per Capita'); 
plt.title('First 2 Principal Components of Data')

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 3c

What could the first and second principal component roughly represent? Interpret the visualization above and report any findings from the first two principal components given by PCA.

<!--
BEGIN QUESTION
name: q3c
points: 3
manual: True
-->

PC1 and PC2 together both seem to roughly represent cases per capita. We have to look at them together since states with low cases per capita seem to be all located in a single "quadrant" of the plot above rather than a vertical split or horizontal split. States with a high PC1 and high PC2 tend to have lower cases per capita while other combinations of high/low values for PC1 and PC2 all seem to have a relatively high proportion of cases per capita.

<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

### Question 3d

Construct a scree plot displaying the proportion of variance captured by each principal component.

<!--
BEGIN QUESTION
name: q3d
points: 3
manual: True
-->

In [ ]:
#Calculations of proportions of variance taken directly from hw10
sns.set(rc = {'figure.figsize': (8,5)})
plt.plot([i for i in range(1, len(s) + 1)], s**2 / sum(s**2)); 
plt.xticks([i for i in range(1, len(s) + 1)], [i for i in range(1, len(s) + 1)]);

plt.xlabel('PC #'); 
plt.ylabel('Fraction of Variance Explained'); 
plt.title('Fraction of Variance Explained by each Principal Component'); 

<!-- END QUESTION -->

### Question 3e

Using the 2D representation of each state's first two principal components $x_i = [\text{PC}_1, \text{PC}_2]$, find each state's 5 closest principal component neighbors using Euclidean distance as the metric. In other words, for each state $i$ with first two principal components $x_i$, the following would be its *closest* principal component neighbor:

$$
\arg \min_{j \ne i} ||x_j - x_i||_2
$$

Store each state's 5 closest neighbors in a Pandas DataFrame, where the index is the state's name and the columns are the 5 closest PC neighbors.

<!--
BEGIN QUESTION
name: q3e
points: 3
-->

In [ ]:
def calc_12(df, state1, state2): 
    state1_row = df[df['state'] == state1]
    state2_row = df[df['state'] == state2]
    state1_pc1 = state1_row.iloc[0]['pc1']
    state1_pc2 = state1_row.iloc[0]['pc2']
    state2_pc1 = state2_row.iloc[0]['pc1']
    state2_pc2 = state2_row.iloc[0]['pc2']
    squared_euc_dist = (state2_pc1 - state1_pc1) ** 2 + (state2_pc2 - state1_pc2) ** 2
    return squared_euc_dist ** 0.5

In [ ]:
#Iterate over all the states for their closest neighbors
results = [[] for i in range(5)]

for state1 in X_pcs['state']:
    l2_scores = []
    for state2 in X_pcs['state']: 
        if state1 != state2: 
            l2_score = calc_12(X_pcs, state1, state2)
            l2_scores.append((state2, l2_score))
    l2_scores.sort(key=lambda x: x[1])
    for i in range(5): 
        results[i].append(l2_scores[i][0])
top_5_closest = pd.DataFrame({'state': list(X_pcs['state']), 
                              '1': results[0], 
                              '2': results[1], 
                              '3': results[2], 
                              '4': results[3], 
                              '5': results[4]})

In [ ]:
state_mins = top_5_closest.set_index('state')
state_mins.head(10)

In [ ]:
grader.check("q3e")

<!-- BEGIN QUESTION -->

### Question 3f

Investigate the principal component neighbors for two states of your choice, and remark on any observations you have about their COVID-19 statistics given in the dataset, geographical or political ties. 

Compare these neighbors with the rankings from the visualization from Question 2c that depicted recent greatest increases in cases across states.

<!--
BEGIN QUESTION
name: q3f
points: 3
manual: True
-->

By observing the PC neighbors of Alabama, we can see that they all have relatively higher proportions of covid cases per capita. Also, geographically, Alabama's PC neighbors are also all close to Alabama, with Mississipi and Tennessee directly bordering Alabama. Also, similar to Alabama, all of its PC neighbors are generally more conservative leaning states. 

By observing the PC neighbors of California, we can see that they all have slightly above average proportions of covid cases per capita. However, unlike Alabama, not all of California's PC neighbors are geographically close to California. For instance, while Arizona and Nevada are close to California, Texas, Pennsylvania, and Delaware are not geographically similar at all to California. Also, unlike Alabama, the political affiliations of California's PC neighbors are not homogenous. For example, California and Delaware are generally democratic-leaning, however, states like Texas is usually republican and Arizona and Pennsylvania are swing states. 

If we compare the rankings from 2c, then we see that for Alabama, a lot of its closest PC neighbors are somewhat near it in terms of rankings in increases in cases across states. This is also somewhat the case with California, with the exception being Delaware with Delaware being quite far from California in terms of rankings. What's quite interesting is that Delaware is closest to California in terms of PC neighbors.


<!-- END QUESTION -->

### Question 3g

We will investigate the relationship between geographical distance in miles and top two principal component distance between any two unique pairs of states. 

For the sake of simplicity, may assume that the singular geographical location of a state is given by the mean latitude and longitude of all the counties in that state. For each unique pair of states, calculate the geographical distance and the Euclidean distance between their top 2 principal components between them. Plot the relationship where the x-axis represents the top 2 principal component distance and the y-axis represents geographical distance.

*Hint*: `geopy` has a function that can calculate distance between two pairs of latitude and longitude!

*Hint*: You should be plotting 1,275 points.

<!--
BEGIN QUESTION
name: q3g
points: 3
-->

In [ ]:
from geopy import distance

#reassign long and lat 
X_pcs['long'] = list(by_state['Long__y'])
X_pcs['lat'] = list(by_state['Lat_y'])

#Looping through all pairs of states
calculated = set()
x, y = [], []
for state1 in X_pcs['state']: 
    for state2 in X_pcs['state']: 
        if state1 != state2: 
            if (state1, state2) in calculated or (state2, state1) in calculated: 
                continue
            calculated.add((state1, state2))
            pc_dist = calc_12(X_pcs, state1, state2)
            state1_row = X_pcs[X_pcs['state'] == state1]
            state2_row = X_pcs[X_pcs['state'] == state2]
            state1_lon = state1_row.iloc[0]['long']
            state1_lat = state1_row.iloc[0]['lat']
            state2_lon = state2_row.iloc[0]['long']
            state2_lat = state2_row.iloc[0]['lat']
            coords_1 = (state1_lat, state1_lon)
            coords_2 = (state2_lat, state2_lon)
            geo_dist = distance.distance(coords_1, coords_2).miles
            x.append(pc_dist)
            y.append(geo_dist)


In [ ]:

ax = sns.scatterplot(x = x, y = y); 
plt.xlabel('Principcal Component Distance'); 
plt.ylabel('Geographic Distance'); 
plt.title('Principcal Component Distance vs. Geographic Distance'); 

### Question 3h

Interpret the relationship displayed in part (g). What does this suggest about the role that geography plays within this dataset despite no geographical information explicitly encoded within the matrix  $X$ from part (a)?

<!--
BEGIN QUESTION
name: q3h
points: 3
-->

Looking at this plot, there appears to be a positive linear relationshiop between geographic distance and principal component distance. From this observation, we can infer that geography is correlated to the features we have in our X matrix (vaccination rate, mask usage, cases). However, this result isn't very surprising given the cultural and political differences between all the states. However, I do think it is interesting that we can analyze the effect of geographic distance, despite there not being any data about geographic distance in the data. 

<!-- BEGIN QUESTION -->

<br/><br/><br/>

---

<br/><br/><br/>

## Question 4: Open EDA

Perform EDA of your choice on the data to generate 2 visualizations. You may follow the line of investigation in the guided sections by visualizing the relationship between safety protocols and the spread of COVID-19 spatially or temporally, or you may explore other areas of your choice. 

For each visualization, make sure to address the following:

1. Address a relationship in the data concerning the spread, effect, or prevention of COVID-19 through a visualization, with at *most* one univariate data visualization. If needed, use an unsupervised learning technique such as PCA to reveal patterns within the data.
2. Comment on the visualization in terms of the distribution, trends, and patterns it shows.
3. Comment on what the visualization indicates about the data with regards to the features you may choose for a supervised learning task.

<!--
BEGIN QUESTION
name: q4
points: 30
manual: True
-->

In [ ]:


#A) This visualization attempts to show whether or not there's a relationship between latitude and median change in daily covid
#cases from 1/22/20 - 9/12/21 Essentially, we are looking to see if there is a relationship between climate and increases in 
#covid cases per state (using latitude as a proxy for climate)

#B) From the scatterplot, we can see a slight negative correlation between latitude and median change in daily covid cases from
#1/22/20 - 9/12/21

#C) From this visualization, we could run a linear regression to help predict changes in covid cases. 



lat_grouped = county_data[['median_increase', 'Lat', 'STNAME']].groupby('STNAME').mean()
sns.scatterplot(data = lat_grouped, x = 'median_increase', y = 'Lat', palette = 'RdBu_r', s = 50)
x = lat_grouped['median_increase']
y = lat_grouped['Lat']
z = np.polyfit(x, y, 1)
p = np.poly1d(z)
plt.plot(x,p(x),"r--")
plt.title('Latitude vs. Median Increase in Covid Cases Per State')

In [ ]:
#A) For this plot, we  wanted to investigate possible trends in mask usage in the counties in the data set.

#B) From this box plot, we can see that the largest range in mask usage was in the ‘always’ category. This maybe reflects the 
#huge differences in attitude towards covid across the nation - with some counties always wearing masks and others not so much.
#Despite this, the four other mask usage categories displayed somewhat similar distributions. Additionally, we can see that in 
#all the mask usages except 'always', there are many outliers. 

#C) From this plot, we could possibly use the types of mask usage in a future supervised learning model, such as classification. 


sns.boxplot(data=mask_use.drop(['COUNTYFP'], axis=1))
plt.title('Proportion of Mask Usage in Dataset')
plt.xlabel('Frequency of Each Mask Usage')
plt.ylabel('Proportion of Types of Mask Usage')

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export()